In [1]:
import pandas as pd
# !pip install openpyxl
# Import data directly from website

excel_file = 'Data_Tables_LGA_Criminal_Incidents_Year_Ending_March_2022.xlsx'
all_sheets = pd.read_excel(excel_file, sheet_name=None)
sheets = all_sheets.keys()

for sheet_name in sheets:
    sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
    sheet.to_csv("%s.csv" % sheet_name, index=False)

Contents	
Table 01

* Criminal incidents and rate per 100,000 population by police region and local government area - April 2012 to March 2022

Table 02	

* Criminal incidents and rate per 100,000 population by principal offence, local government area and police service area - April 2012 to March 2022

Table 03	

* Criminal incidents by principal offence, local government area and postcode or suburb/town - April 2012 to March 2022

Table 04	

* Criminal incidents by location type and local government area - April 2012 to March 2022

Table 05	

* Criminal incidents by charge status and local government area - April 2012 to March 2022
	
### Data exclusions	
Table 01 

* excludes criminal incidents where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.	

Tables 02 - 05 

* excludes criminal incidents at Justice institutions and immigration facilities, Unincorporated Victoria and where the geographic location is unknown. For further information of these geographic locations please refer to the Explanatory Notes and Glossary on the website.	
	
Notes	
Recorded crime statistics are based on data extracted by Victoria police on the 18th day after the reference period, and are subject to movement between releases. For more information about how statistics are compiled, refer to the Explanatory notes on the CSA website.	
In order to maintain confidentiality, sensitive offence counts for subdivisions 'A10 Homicide and related offences' and 'A30 Sexual offences' with a value of 3 or less are given a value of 2 to calculate totals.	
Rates are based on populations provided by the Australian Bureau of Statistics (ABS). The most recent year of data was not available from the ABS in time for it to be used to calculate current year rates. The CSA uses estimates created by the Victorian State Government 'Victoria in Futures' report. 	
This work is licenced under a Creative Commons Attribution 4.0 International License. When reporting this data you must attribute the Crime Statistics Agency (or CSA) as the source.

In [2]:
#Create list of dataframes from csv files
file_list = []
for i in range(1,6):
    file_list.append(f"Table 0{i}.csv")

df_list = [pd.read_csv(file) for file in file_list]

In [3]:
file_list

['Table 01.csv',
 'Table 02.csv',
 'Table 03.csv',
 'Table 04.csv',
 'Table 05.csv']

In [4]:
df_list = [pd.read_csv(file) for file in file_list]

In [ ]:
# df_list[2].info()

In [6]:
# df_list[0]['Date'] = df_list[0][]
# police_region_df = df_list[0].loc[df_list[0].Year > 2018].drop('Year ending',axis=1)
# police_region_df.head()
police_region_df[police_region_df["Local Government Area"]=="Ballarat"]

,Year,Police Region,Local Government Area,Incidents Recorded,"Rate per 100,000 population"


In [7]:
police_region_df["Local Government Area"] = police_region_df["Local Government Area"].str.strip()

In [8]:
police_region_df.drop(columns=["Incidents Recorded", "Rate per 100,000 population"], inplace= True)


In [8]:
police_region_df.dtypes

Year                             int64
Police Region                   object
Local Government Area           object
Incidents Recorded               int64
Rate per 100,000 population    float64
dtype: object

In [6]:
# Remove data fields that are repated in data set that has the postcodes 
police_region_df.head()

,Year,Police Region,Local Government Area,Incidents Recorded,"Rate per 100,000 population"
0,2022,1 North West Metro,Banyule,5244,4086.865525
1,2022,1 North West Metro,Brimbank,12330,6200.959480
2,2022,1 North West Metro,Darebin,9276,5751.475727
3,2022,1 North West Metro,Hobsons Bay,4599,4804.896128
4,2022,1 North West Metro,Hume,12196,4919.684615


In [16]:
police_division_df = df_list[1].loc[df_list[1].Year > 2018].drop('Year ending',axis=1)
police_division_df.head()

,Year,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,"PSA Rate per 100,000 population","LGA Rate per 100,000 population"
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137


In [13]:
police_division_df.dtypes 

Year                                 int64
Police Service Area                 object
Local Government Area               object
Offence Division                    object
Offence Subdivision                 object
Offence Subgroup                    object
Incidents Recorded                   int64
PSA Rate per 100,000 population    float64
LGA Rate per 100,000 population    float64
dtype: object

In [14]:
# Remove data fields that are repated in data set that has the postcodes 
# police_division_df .drop(columns=["Offence Division","Offence Subdivision", 
#                                         "Offence Subgroup","Incidents Recorded"], inplace= True)

In [15]:
police_data_df = pd.merge(police_division_df, police_region_df, on=["Year", "Local Government Area"], how = "left")
police_data_df.head()

,Year,Police Service Area,Local Government Area,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded_x,"PSA Rate per 100,000 population","LGA Rate per 100,000 population",Police Region,Incidents Recorded_y,"Rate per 100,000 population"
0,2022,Ballarat,Ballarat,A Crimes against the person,A10 Homicide and related offences,A10 Homicide and related offences,5,4.059101,4.328089,NaN,NaN,NaN
1,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A211 FV Serious assault,120,97.418429,103.874128,NaN,NaN,NaN
2,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A212 Non-FV Serious assault,147,119.337575,127.245807,NaN,NaN,NaN
3,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,"A22 Assault police, emergency services or othe...",53,43.026473,45.877740,NaN,NaN,NaN
4,2022,Ballarat,Ballarat,A Crimes against the person,A20 Assault and related offences,A231 FV Common assault,222,180.224093,192.167137,NaN,NaN,NaN


In [ ]:
police_data_df.dtypes

In [37]:
postcode_df = df_list[2].loc[df_list[2].Year > 2018].drop('Year ending',axis=1)
postcode_df.head()

,Year,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded
0,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2
1,2022,Alpine,3691,Dederang,B Property and deception offences,B30 Burglary/Break and enter,B321 Residential non-aggravated burglary,1
2,2022,Alpine,3691,Dederang,B Property and deception offences,B50 Deception,B53 Obtain benefit by deception,1
3,2022,Alpine,3691,Dederang,D Public order and security offences,D20 Disorderly and offensive conduct,D22 Drunk and disorderly in public,1
4,2022,Alpine,3691,Kergunyah South,E Justice procedures offences,E10 Justice procedures,E14 Pervert the course of justice or commit pe...,1


In [38]:
all_crime_data_df = pd.merge(postcode_df, police_data_df, on=["Year", "Local Government Area"])
all_crime_data_df.head()

,Year,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Police Service Area,"PSA Rate per 100,000 population","LGA Rate per 100,000 population",Police Region
0,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,2.745170,15.241518,NaN
1,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,15.098436,83.828350,NaN
2,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,12.353266,68.586832,NaN
3,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,1.372585,7.620759,NaN
4,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,21.961362,121.932145,NaN
5,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,17.843606,99.069868,NaN
6,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,26.079117,144.794422,NaN
7,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,1.372585,7.620759,NaN
8,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,1.372585,7.620759,NaN
9,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,1.372585,7.620759,NaN


In [39]:
#Convert dataframe to csv

all_crime_data_df.to_csv("all_crime_data.csv")

In [41]:
csv_file = "all_crime_data.csv"
all_crime_data_df = pd.read_csv(csv_file)
all_crime_data_df.head()

,Unnamed: 0,Year,Local Government Area,Postcode,Suburb/Town Name,Offence Division,Offence Subdivision,Offence Subgroup,Incidents Recorded,Police Service Area,"PSA Rate per 100,000 population","LGA Rate per 100,000 population",Police Region
0,0,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,2.745170,15.241518,NaN
1,1,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,15.098436,83.828350,NaN
2,2,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,12.353266,68.586832,NaN
3,3,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,1.372585,7.620759,NaN
4,4,2022,Alpine,3691,Dederang,A Crimes against the person,Other crimes against the person,Other crimes against the person,2,Wangaratta,21.961362,121.932145,NaN


In [44]:
from sqlalchemy import create_engine

In [45]:
#connect to local database
rds_connection_string = "postgres:Fedelma22!@localhost:5432/Project3"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [47]:
# engine.table_names()

In [ ]:
all_crime_data_df.to_sql(name='CRIME_DATA, con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from CRIME DATA', con=engine).head()